In [305]:
import configparser 
import logging
import datetime as dt
from influxdb import InfluxDBClient, DataFrameClient
import json
import pandas as pd

In [306]:
config = configparser.ConfigParser()
config.read('./configs/monitor.conf')

['./configs/monitor.conf']

In [307]:
def get_influxdb_clients():
    # FIXME
    influxdb_client = InfluxDBClient(
        config.get('INFLUXDB', 'ADDRESS'),
        config.get('INFLUXDB', 'PORT'),
        config.get('INFLUXDB', 'USER'),
        config.get('INFLUXDB', 'PASSWORD'),
        config.get('INFLUXDB', 'DB_NAME'))

    influxdb_dataframe_client = DataFrameClient(
        config.get('INFLUXDB', 'ADDRESS'),
        config.get('INFLUXDB', 'PORT'),
        config.get('INFLUXDB', 'USER'),
        config.get('INFLUXDB', 'PASSWORD'),
        config.get('INFLUXDB', 'DB_NAME'))

    return influxdb_client, influxdb_dataframe_client

In [308]:
uuid = '100316824873160941010780204477983883413'


def get_influxdb_data(client, df_client):

    last_element_query = 'select * from image order by desc limit 1'
    last_result = client.query(last_element_query)
    
    last_uuid = list(last_result.get_points())[0]['uuid']
    
    data_query = 'select * from image where uuid = \'{}\' group by "FILTER"  order by asc'.format(
        last_uuid)
    
#     data_query = 'select * from image group by "FILTER" order by asc limit 100'.format(
#                      uuid
#                     )
    result = df_client.query(data_query)
    result = dict(
        [(k[1][0][1], v.reset_index(level=0).to_json()) for k, v in result.items()])

    return last_result, result


In [309]:
client, df_client = get_influxdb_clients()

In [310]:
last_result, result = get_influxdb_data(client, df_client)

In [323]:
result

{'R': '{"index":{"0":1557621579927,"1":1557621585740,"2":1557621591653},"BACKGROUND":{"0":4804.625,"1":4767.313,"2":4752.654},"DEC_CENTER":{"0":36.935891,"1":36.93572,"2":36.935945},"DEC_HDR":{"0":36.935278,"1":36.935278,"2":36.935278},"EXPTIME":{"0":60,"1":60,"2":60},"FLUX_MAX":{"0":2734.683,"1":2572.294,"2":3034.291},"FWHM_IMAGE":{"0":2.53,"1":2.54,"2":2.31},"NUMBER":{"0":125,"1":126,"2":129},"OBJECT":{"0":"j0713","1":"j0713","2":"j0713"},"OBSERVER":{"0":"MD","1":"MD","2":"MD"},"PHOTOMETRY_STATUS":{"0":1,"1":1,"2":1},"RA_CENTER":{"0":108.290493,"1":108.290428,"2":108.290489},"RA_HDR":{"0":108.289583,"1":108.289583,"2":108.289583},"SNR_WIN":{"0":80.42,"1":83.9,"2":86.79},"X_IMAGE":{"0":514.7047,"1":514.6284,"2":514.7081},"Y_IMAGE":{"0":514.0812,"1":513.6413,"2":514.2977},"image_time":{"0":"2019-03-19T22:39:33.49","1":"2019-03-19T22:40:35.09","2":"2019-03-19T22:41:36.72"},"uuid":{"0":"225478220844960246434850379525829752633","1":"225478220844960246434850379525829752633","2":"2254782208

In [322]:
value.columns

Index(['index', 'BACKGROUND', 'DEC_CENTER', 'DEC_HDR', 'EXPTIME', 'FLUX_MAX',
       'FWHM_IMAGE', 'NUMBER', 'OBJECT', 'OBSERVER', 'PHOTOMETRY_STATUS',
       'RA_CENTER', 'RA_HDR', 'SNR_WIN', 'X_IMAGE', 'Y_IMAGE', 'image_time',
       'uuid'],
      dtype='object')

In [276]:
for k, v in result.items():
    a = pd.read_json(v)

In [327]:
ra_object, dec_object, ra_center, dec_center = (108.28958, 36.9352,
                                                108.290493, 36.8352)

In [328]:
from astropy.coordinates import SkyCoord

In [329]:
obj_coo = SkyCoord(ra_object, dec_object, unit='deg')
center_coo = SkyCoord(ra_center, dec_center, unit='deg')

In [338]:
sep = obj_coo.separation(center_coo)

In [341]:
sep.arcmin

6.0001599788845255